In [8]:
import gym
import numpy as np
import math

import time

In [9]:
env = gym.make("CartPole-v1")

In [10]:
# run one episode 
def run_episode(environment, policy = None, render = False):
    obs = env.reset()
    total_reward = 0
    step_index = 0
    for i in range(1000):
        if render:
            environment.render()
        if policy is None:
            action = environment.action_space.sample()
        else:
            a,b = observationState(environment, obs)
            action = policy[a][b]
        obs, reward, done, _ = env.step(action)
        total_reward += gamma ** step_index * reward
        step_index += 1
        if done:
            break
    return total_reward

In [11]:
def observationState(environment, obs):
    env_low = environment.observation_space.low
    env_high = environment.observation_space.high
    env_dx = (env_high - env_low) / n_states
    a = int((obs[0] - env_low[0])/env_dx[0])
    b = int((obs[1] - env_low[1])/env_dx[1])
    return a, b

def discretize(obs, env):
    bucket = [1, 1, 6, 12]
    upper_bound = env.observation_space.high
    lower_bound = env.observation_space.low
    upper_bound[1] = 0.5
    lower_bound[1] = -0.5
    upper_bound[3] = math.radians(50)
    lower_bound[3] = -math.radians(50)
    newobs = []
    for i in range(len(upper_bound)):
        ratio = (obs[i] + abs(lower_bound[i])) / (upper_bound[i] - lower_bound[i])
        temp = int(round((bucket[i] - 1) * ratio))
        temp = min(bucket[i] - 1, max(0, temp))
        newobs.append(temp)

    return np.array(newobs)

In [14]:
env.seed(0)
np.random.seed(0)
n_states = 46
initial_lr = 1.0 # Learning rate
min_lr = 0.003
gamma = 1.0
eps = 0.02
qTable = np.zeros((n_states, n_states, 3))
for i in range(10000):
    obs = env.reset()
    obs = discretize(obs, env)
    total_reward = 0
    eta = max(min_lr, initial_lr * (0.85 ** (i//100)))
    for j in range(10000):
            a, b = observationState(env, obs)
            if np.random.uniform(0, 1) < eps:
                action = np.random.choice(env.action_space.n)
            else:
                logits = qTable[a][b]
                logits_exp = np.exp(logits)
                probs = logits_exp / np.sum(logits_exp)
                action = np.random.choice(env.action_space.n, p=probs)
            obs, reward, done, _ = env.step(action)
            obs = discretize(obs, env)
            total_reward += reward
            # update q table
            a_, b_ = observationState(env, obs)
            qTable[a][b][action] = qTable[a][b][action] + eta * (reward + gamma *  np.max(qTable[a_][b_]) - qTable[a][b][action])
            if done:
                break
    if i % 100 == 0:
        print('Iteration #%d -- Total reward = %d.' %(i+1, total_reward))
solution_policy = np.argmax(qTable, axis=2)
solution_policy_scores = [run_episode(env, solution_policy, False) for _ in range(100)]
print("Average score of solution = ", np.mean(solution_policy_scores))
# Animate it
run_episode(env, solution_policy, True)

ValueError: a and p must have same size

In [2]:
import gym
import numpy as np
import math

import time
env = gym.make("CartPole-v1")

# run one episode
def run_episode(environment, policy = None, render = False):
    obs = env.reset()
    total_reward = 0
    step_index = 0
    for i in range(1000):
        if render:
            environment.render()
        if policy is None:
            action = environment.action_space.sample()
        else:
            a,b = observationState(environment, obs)
            action = policy[a][b]
        obs, reward, done, _ = env.step(action)
        total_reward += gamma ** step_index * reward
        step_index += 1
        if done:
            break
    return total_reward

def observationState(environment, obs):
    env_low = environment.observation_space.low
    env_high = environment.observation_space.high
    env_dx = (env_high - env_low) / n_states
    a = int((obs[0] - env_low[0])/env_dx[0])
    b = int((obs[1] - env_low[1])/env_dx[1])
    return a, b

def discretize(obs, env):
    bucket = [1, 1, 6, 12]
    upper_bound = env.observation_space.high
    lower_bound = env.observation_space.low
    upper_bound[1] = 0.5
    lower_bound[1] = -0.5
    upper_bound[3] = math.radians(50)
    lower_bound[3] = -math.radians(50)
    newobs = []
    for i in range(len(upper_bound)):
        ratio = (obs[i] + abs(lower_bound[i])) / (upper_bound[i] - lower_bound[i])
        temp = int(round((bucket[i] - 1) * ratio))
        temp = min(bucket[i] - 1, max(0, temp))
        newobs.append(temp)

    return np.array(newobs)

env.seed(0)
np.random.seed(0)
n_states = 30000
initial_lr = 0.09 # Learning rate
min_lr = 0.003
gamma = 1.0
eps = 0.1
qTable = np.zeros((n_states, n_states, 2))
for i in range(1000):
    obs = env.reset()
    obs = discretize(obs, env)
    total_reward = 0
    eta = max(min_lr, initial_lr * (0.85 ** (i//100)))
    for j in range(1000):
            a, b = observationState(env, obs)
            if np.random.uniform(0, 1) < eps:
                action = np.random.choice(env.action_space.n)
            else:
                logits = qTable[a][b]
                logits_exp = np.exp(logits)
                probs = logits_exp / np.sum(logits_exp)
                action = np.random.choice(env.action_space.n, p=probs)
            obs, reward, done, _ = env.step(action)
            obs = discretize(obs, env)
            total_reward += reward
            # update q table
            a_, b_ = observationState(env, obs)
            qTable[a][b][action] = qTable[a][b][action] + eta * (reward + gamma *  np.max(qTable[a_][b_]) - qTable[a][b][action])
            if done:
                break
    if i % 100 == 0:
        print('Iteration #%d -- Total reward = %d.' %(i+1, total_reward))
solution_policy = np.argmax(qTable, axis=2)
solution_policy_scores = [run_episode(env, solution_policy, False) for _ in range(100)]
print("Average score of solution = ", np.mean(solution_policy_scores))
# Animate it
run_episode(env, solution_policy, True)

Iteration #1 -- Total reward = 11.
Iteration #101 -- Total reward = 9.
Iteration #201 -- Total reward = 17.
Iteration #301 -- Total reward = 9.
Iteration #401 -- Total reward = 10.
Iteration #501 -- Total reward = 9.
Iteration #601 -- Total reward = 12.
Iteration #701 -- Total reward = 10.
Iteration #801 -- Total reward = 9.
Iteration #901 -- Total reward = 11.


IndexError: index -32880 is out of bounds for axis 0 with size 30000

In [3]:
import gym
import numpy as np
import math
from collections import deque

class QCartPoleSolver():
    def __init__(self, buckets=(1, 1, 6, 12,), n_episodes=1000, n_win_ticks=200, min_alpha=0.1, min_epsilon=0.1, gamma=1.0, ada_divisor=25, max_env_steps=None, quiet=False, monitor=False):
        self.buckets = buckets # down-scaling feature space to discrete range
        self.n_episodes = n_episodes # training episodes 
        self.n_win_ticks = n_win_ticks # average ticks over 100 episodes required for win
        self.min_alpha = min_alpha # learning rate
        self.min_epsilon = min_epsilon # exploration rate
        self.gamma = gamma # discount factor
        self.ada_divisor = ada_divisor # only for development purposes
        self.quiet = quiet

        self.env = gym.make('CartPole-v1')
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps
        #if monitor: self.env = gym.wrappers.Monitor(self.env, 'tmp/cartpole-1', force=True) # record results for upload

        self.Q = np.zeros(self.buckets + (self.env.action_space.n,))

    def discretize(self, obs):
        upper_bounds = [self.env.observation_space.high[0], 0.5, self.env.observation_space.high[2], math.radians(50)]
        lower_bounds = [self.env.observation_space.low[0], -0.5, self.env.observation_space.low[2], -math.radians(50)]
        ratios = [(obs[i] + abs(lower_bounds[i])) / (upper_bounds[i] - lower_bounds[i]) for i in range(len(obs))]
        new_obs = [int(round((self.buckets[i] - 1) * ratios[i])) for i in range(len(obs))]
        new_obs = [min(self.buckets[i] - 1, max(0, new_obs[i])) for i in range(len(obs))]
        return tuple(new_obs)

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.Q[state])

    def update_q(self, state_old, action, reward, state_new, alpha):
        self.Q[state_old][action] += alpha * (reward + self.gamma * np.max(self.Q[state_new]) - self.Q[state_old][action])

    def get_epsilon(self, t):
        return max(self.min_epsilon, min(1, 1.0 - ((t + 1) / self.ada_divisor)))
        #return max(self.min_epsilon, 1)

    def get_alpha(self, t):
        return max(self.min_alpha, min(1.0, 1.0 - ((t + 1) / self.ada_divisor)))
        #return max(self.min_alpha, 1)

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            current_state = self.discretize(self.env.reset())

            alpha = self.get_alpha(e)
            epsilon = self.get_epsilon(e)
            done = False
            i = 1

            while not done:
                self.env.render()
                action = self.choose_action(current_state, epsilon)
                obs, reward, done, _ = self.env.step(action)
                new_state = self.discretize(obs)
                self.update_q(current_state, action, reward, new_state, alpha)
                current_state = new_state
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials :'.format(e, e - 100), " score :", mean_score)
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

        if not self.quiet: print('Did not solve after {} episodes :'.format(e))
        return e

if __name__ == "__main__":
    solver = QCartPoleSolver()
    solver.run()
    # gym.upload('tmp/cartpole-1', api_key='')

[Episode 0] - Mean survival time over last 100 episodes was 19.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 61.12 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 162.81 ticks.
('Ran 260 episodes. Solved after 160 trials :', ' score :', 200.22)
